In [1]:
import glob
import pandas as pd
import numpy as np
import optuna
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
import importlib
import airbnb_test_func_ver2 as tf
importlib.reload(tf)

%matplotlib inline

df = pd.read_csv("train.csv", index_col=0)
df = tf.data_pre(df)

In [2]:
def objective(trial):

    df_train, df_val = train_test_split(df, test_size=0.2)
    
    col = "y"
    train_y = df_train[col]
    train_x = df_train.drop(col, axis=1)

    val_y = df_val[col]
    val_x = df_val.drop(col, axis=1)

    trains = lgb.Dataset(train_x, train_y)
    valids = lgb.Dataset(val_x, val_y)

    #https://lightgbm.readthedocs.io/en/latest/Parameters.html
    #https://knknkn.hatenablog.com/entry/2021/06/29/125226
    #https://zenn.dev/megane_otoko/articles/2021ad_09_optuna_optimization

    learning_rate = trial.suggest_float('learning_rate', 0.0, 1.0)
    num_leaves =  trial.suggest_int("num_leaves", 2, 20)
    tree_learner = trial.suggest_categorical('tree_learner', ["serial", "feature", "data", "voting"])
    lambda_l1= trial.suggest_float("lambda_l1", 0.0, 200.0)
    lambda_l2 = trial.suggest_float("lambda_l2", 0.0, 200.0)
    max_depth = trial.suggest_int("max_depth", 2, 20)
    
    params = {
        "task": "train", 
        "objective": "regression",
        "boosting_type": "gbdt", 
        "metrics": {"rmse"}, 
        "learning_rate": learning_rate, 
        "num_leaves": num_leaves, 
        "tree_learner": tree_learner,
        "lambda_l1": lambda_l1, 
        "lambda_l2": lambda_l2, 
        "seed": 17, 
        "max_depth": max_depth
        
        
    }

        #カテゴリカルデータをリストで渡す
    categorical_list = [
                        "cleaning_fee",
                        "host_identity_verified",
                        "cancellation_policy",
                        "room_type",
                        "neighbourhood",
                        "zipcode" 
                        ]
    #https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.train.html
    #https://lightgbm.readthedocs.io/en/latest/Python-API.html
    model = lgb.train(params, 
                        trains, 
                        valid_sets=valids, 
                        categorical_feature=categorical_list, 
                        num_boost_round=1000, 
                        early_stopping_rounds=50, 
                        )

    pred_y = model.predict(val_x)
    rmse = np.sqrt(mse(val_y, pred_y))
    
    return rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

print("Best trial: {}".format(study.best_trial.number))
print("Best params: {}".format(study.best_trial.params))
print("Best score: {}".format(study.best_trial.value)) 

[I 2023-05-20 21:11:42,335] A new study created in memory with name: no-name-b3f8524d-fb94-4e93-b372-ff39b8c266bd
C:\ProgramData\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\ProgramData\anaconda3\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cancellation_policy', 'cleaning_fee', 'host_identity_verified', 'neighbourhood', 'room_type', 'zipcode']
  _log_warning('categorical_feature in Dataset is overridden.\n'


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006045 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1893
[LightGBM] [Info] Number of data points in the train set: 44466, number of used features: 16
[LightGBM] [Info] Start training from score 160.452548
[1]	valid_0's rmse: 144.655
Training until validation scores don't improve for 50 rounds
[2]	valid_0's rmse: 134.963
[3]	valid_0's rmse: 129.049
[4]	valid_0's rmse: 125.532
[5]	valid_0's rmse: 123.039
[6]	valid_0's rmse: 120.619
[7]	valid_0's rmse: 119.513
[8]	valid_0's rmse: 118.23
[9]	valid_0's rmse: 117.607
[10]	valid_0's rmse: 117.204
[11]	valid_0's rmse: 116.906
[12]	valid_0's rmse: 116.43
[13]	valid_0's rmse: 116.075
[14]	valid_0's rmse: 115.525
[15]	valid_0's rmse: 115.228
[16]	valid_0's rmse: 115.12
[17]	valid_0's rmse: 115.017
[18]	valid_0's rmse: 114.93
[19]	valid_0's rmse: 114.64
[20]	valid_0's rmse: 114.406
[21]	valid_0's rmse: 113.9

C:\ProgramData\anaconda3\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\ProgramData\anaconda3\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[106]	valid_0's rmse: 111.66
[107]	valid_0's rmse: 111.644
[108]	valid_0's rmse: 111.617
[109]	valid_0's rmse: 111.625
[110]	valid_0's rmse: 111.622
[111]	valid_0's rmse: 111.629
[112]	valid_0's rmse: 111.6
[113]	valid_0's rmse: 111.595
[114]	valid_0's rmse: 111.616
[115]	valid_0's rmse: 111.631
[116]	valid_0's rmse: 111.657
[117]	valid_0's rmse: 111.674
[118]	valid_0's rmse: 111.676
[119]	valid_0's rmse: 111.655
[120]	valid_0's rmse: 111.649
[121]	valid_0's rmse: 111.631
[122]	valid_0's rmse: 111.611
[123]	valid_0's rmse: 111.597
[124]	valid_0's rmse: 111.626
[125]	valid_0's rmse: 111.618
[126]	valid_0's rmse: 111.663
[127]	valid_0's rmse: 111.68
[128]	valid_0's rmse: 111.663
[129]	valid_0's rmse: 111.658
[130]	valid_0's rmse: 111.608
[131]	valid_0's rmse: 111.672
[132]	valid_0's rmse: 111.631
[133]	valid_0's rmse: 111.647
[134]	valid_0's rmse: 111.66
[135]	valid_0's rmse: 111.652
[136]	valid_0's rmse: 111.651
[137]	valid_0's rmse: 111.588
[138]	valid_0's rmse: 111.601
[139]	valid_0's

[I 2023-05-20 21:11:43,065] Trial 0 finished with value: 111.58130511063099 and parameters: {'learning_rate': 0.4330434988970856, 'num_leaves': 6, 'tree_learner': 'data', 'lambda_l1': 121.0266990353546, 'lambda_l2': 38.582505652208134, 'max_depth': 3}. Best is trial 0 with value: 111.58130511063099.
C:\ProgramData\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\ProgramData\anaconda3\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cancellation_policy', 'cleaning_fee', 'host_identity_verified', 'neighbourhood', 'room_type', 'zipcode']
  _log_warning('categorical_feature in Dataset is overridden.\n'


...

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007349 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1889
[LightGBM] [Info] Number of data points in the train set: 44466, number of used features: 16
[LightGBM] [Info] Start training from score 160.196397
[1]	valid_0's rmse: 148.37
Training until validation scores don't improve for 50 rounds
[2]	valid_0's rmse: 137.31
[3]	valid_0's rmse: 130.083
[4]	valid_0's rmse: 125.284
[5]	valid_0's rmse: 122.011
[6]	valid_0's rmse: 119.733
[7]	valid_0's rmse: 118.249
[8]	valid_0's rmse: 116.787
[9]	valid_0's rmse: 115.77
[10]	valid_0's rmse: 115.072
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[11]	valid_0's rmse: 114.153
[12]	valid_0's rmse: 113.01
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[13]	valid_0's rmse: 112.722
[14]	valid_0's rmse: 111.87
[15]	valid_0's rmse: 111.661
[16]	valid_0's rmse: 

[I 2023-05-20 21:13:06,592] Trial 98 finished with value: 108.93598236482917 and parameters: {'learning_rate': 0.32555619430930666, 'num_leaves': 16, 'tree_learner': 'voting', 'lambda_l1': 118.46587701207575, 'lambda_l2': 116.02756152935586, 'max_depth': 5}. Best is trial 7 with value: 104.21660216230514.
C:\ProgramData\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\ProgramData\anaconda3\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cancellation_policy', 'cleaning_fee', 'host_identity_verified', 'neighbourhood', 'room_type', 'zipcode']
  _log_warning('categorical_feature in Dataset is overridden.\n'


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003159 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1896
[LightGBM] [Info] Number of data points in the train set: 44466, number of used features: 16
[LightGBM] [Info] Start training from score 159.729928
[1]	valid_0's rmse: 147.549
Training until validation scores don't improve for 50 rounds
[2]	valid_0's rmse: 134.941
[3]	valid_0's rmse: 128.377
[4]	valid_0's rmse: 124.219
[5]	valid_0's rmse: 121.193
[6]	valid_0's rmse: 118.473
[7]	valid_0's rmse: 117.235
[8]	valid_0's rmse: 115.971
[9]	valid_0's rmse: 115.156
[10]	valid_0's rmse: 114.74
[11]	valid_0's rmse: 114.238
[12]	valid_0's rmse: 113.315
[13]	valid_0's rmse: 112.936
[14]	valid_0's rmse: 112.629
[15]	valid_0's rmse: 112.46
[16]	valid_0's rmse: 112.379
[17]	valid_0's rmse: 112.25
[18]	valid_0's rmse: 112.166
[19]	valid_0's rms

C:\ProgramData\anaconda3\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\ProgramData\anaconda3\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')
[I 2023-05-20 21:13:07,288] Trial 99 finished with value: 110.44011591108003 and parameters: {'learning_rate': 0.41193803424176745, 'num_leaves': 13, 'tree_learner': 'data', 'lambda_l1': 102.07521213916071, 'lambda_l2': 124.8022870400302, 'max_depth': 17}. Best is trial 7 with value: 104.21660216230514.


[68]	valid_0's rmse: 110.756
[69]	valid_0's rmse: 110.75
[70]	valid_0's rmse: 110.763
[71]	valid_0's rmse: 110.804
[72]	valid_0's rmse: 110.696
[73]	valid_0's rmse: 110.72
[74]	valid_0's rmse: 110.692
[75]	valid_0's rmse: 110.692
[76]	valid_0's rmse: 110.735
[77]	valid_0's rmse: 110.717
[78]	valid_0's rmse: 110.716
[79]	valid_0's rmse: 110.729
[80]	valid_0's rmse: 110.712
[81]	valid_0's rmse: 110.704
[82]	valid_0's rmse: 110.686
[83]	valid_0's rmse: 110.648
[84]	valid_0's rmse: 110.681
[85]	valid_0's rmse: 110.663
[86]	valid_0's rmse: 110.614
[87]	valid_0's rmse: 110.612
[88]	valid_0's rmse: 110.619
[89]	valid_0's rmse: 110.614
[90]	valid_0's rmse: 110.585
[91]	valid_0's rmse: 110.583
[92]	valid_0's rmse: 110.608
[93]	valid_0's rmse: 110.616
[94]	valid_0's rmse: 110.589
[95]	valid_0's rmse: 110.565
[96]	valid_0's rmse: 110.538
[97]	valid_0's rmse: 110.501
[98]	valid_0's rmse: 110.44
[99]	valid_0's rmse: 110.457
[100]	valid_0's rmse: 110.493
[101]	valid_0's rmse: 110.465
[102]	valid_0's